In [1]:
%reload_ext autoreload
%autoreload 2

In [9]:
import pandas as pd
import os.path as op
import glob

from biomag_single_subject import run_all

from joblib import Parallel, delayed

import mkl
from hcp_central.s3_utils import get_aws_credentials, upload_to_s3
from mne.utils import set_log_file

In [10]:
mkl.set_num_threads(4)
set_log_file('warning')

/home/ubuntu/miniconda2/envs/swish/lib/python2.7/site-packages/ipykernel/__main__.py:2: RuntimeWarning: Log entries will be appended to the file. Use overwrite=False to avoid this message in the future.
  from ipykernel import kernelapp as app


In [11]:
df = pd.read_csv(
    op.join(op.expanduser("~"), "mne-hcp-data",
            "unrel_subject_processing_map.csv")
)

In [12]:
subjects_rs = df.Subject[df.unrelated_rs_source].tolist()

In [13]:
Parallel(n_jobs=16)(delayed(run_all)(str(subject)) for subject in subjects_rs);

/home/ubuntu/github/mne-hcp/hcp/io/read.py:120: RuntimeWarning: Currently direct inclusion of 4D weight tables is not supported. For critical use cases please take into account the MNE command "mne_create_comp_data" to include weights as printed out by the 4D "print_table" routine.
  sort_by_ch_name=False, rename_channels=False, preload=False)
/home/ubuntu/github/mne-hcp/hcp/io/read.py:120: RuntimeWarning: Currently direct inclusion of 4D weight tables is not supported. For critical use cases please take into account the MNE command "mne_create_comp_data" to include weights as printed out by the 4D "print_table" routine.
  sort_by_ch_name=False, rename_channels=False, preload=False)
/home/ubuntu/github/mne-hcp/hcp/io/read.py:120: RuntimeWarning: Currently direct inclusion of 4D weight tables is not supported. For critical use cases please take into account the MNE command "mne_create_comp_data" to include weights as printed out by the 4D "print_table" routine.
  sort_by_ch_name=False, 

[['/mnt2/dynamic-scales/105923/rest-psd-alpha-epochs-run0.hdf5',
  '/mnt2/dynamic-scales/105923/rest-psd-alpha-epochs-run1.hdf5',
  '/mnt2/dynamic-scales/105923/rest-psd-alpha-epochs-run2.hdf5',
  '/mnt2/dynamic-scales/105923/rest-psd-broadband-epochs-run0.hdf5',
  '/mnt2/dynamic-scales/105923/rest-psd-broadband-epochs-run1.hdf5',
  '/mnt2/dynamic-scales/105923/rest-psd-broadband-epochs-run2.hdf5'],
 ['/mnt2/dynamic-scales/111514/rest-psd-alpha-epochs-run0.hdf5',
  '/mnt2/dynamic-scales/111514/rest-psd-alpha-epochs-run1.hdf5',
  '/mnt2/dynamic-scales/111514/rest-psd-alpha-epochs-run2.hdf5',
  '/mnt2/dynamic-scales/111514/rest-psd-broadband-epochs-run0.hdf5',
  '/mnt2/dynamic-scales/111514/rest-psd-broadband-epochs-run1.hdf5',
  '/mnt2/dynamic-scales/111514/rest-psd-broadband-epochs-run2.hdf5'],
 ['/mnt2/dynamic-scales/116524/rest-psd-alpha-epochs-run0.hdf5',
  '/mnt2/dynamic-scales/116524/rest-psd-alpha-epochs-run1.hdf5',
  '/mnt2/dynamic-scales/116524/rest-psd-alpha-epochs-run2.hdf5',

In [14]:
aws_up_key, aws_up_secret = get_aws_credentials(
    op.join(op.expanduser("~"), "mne-hcp-aws", 'aws_dynamic_scales.csv'))
for fname in glob.glob('/mnt2/dynamic-scales/*/*hdf5'):
    key = op.join('hcp-meg', '/'.join(fname.split('/')[-2:]))
    upload_to_s3(aws_key=aws_up_key,
                 aws_secret=aws_up_secret, bucket='dynamic-scales', key=key,
                 fname=fname)